In [ ]:
!nvidia-smi

Mon Mar 20 12:40:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  1.13 ; cuda:  cu116


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install YOLOv8

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.54 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 25.5/78.2 GB disk)


## Install Supervision

In [ ]:
!pip install supervision==0.2.1


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.2.1


##DOWNLOAD VIDEO

In [ ]:
!gdown --id 1H1wCn-_IviEOEpoGgb99EbTCBiHAkgl7

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1H1wCn-_IviEOEpoGgb99EbTCBiHAkgl7
To: /content/video1611_11.mp4
100% 176M/176M [00:02<00:00, 73.9MB/s]


In [ ]:
SOURCE_VIDEO_PATH = f"{HOME}/input.mp4"
TARGET_VIDEO_PATH = f"{HOME}/output/output.mp4"


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

  0%|          | 0.00/6.23M [00:00<?, ?B/s]

In [ ]:
import supervision as sv

In [ ]:
sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

VideoInfo(width=1920, height=1080, fps=20, total_frames=2771)

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from ultralytics import YOLO
import supervision as sv
import cv2
from supervision import PolygonZone
from IPython import display
from typing import Optional, Tuple

import cv2
import numpy as np

from supervision import Detections
from supervision.detection.utils import generate_2d_mask
from supervision.draw.color import Color
from supervision.draw.utils import draw_polygon, draw_text
from supervision.geometry.core import Position, Point
from supervision.geometry.utils import get_polygon_center

MODEL = "yolov8n.pt"
model = YOLO(MODEL)
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

SOURCE_VIDEO_PATH = f"{HOME}/input.mp4"
TARGET_VIDEO_PATH = f"{HOME}/output/output.mp4"
VIDEO_INFO = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)


WIDTH = VIDEO_INFO.width
HEIGHT = VIDEO_INFO.height
FPS = VIDEO_INFO.fps
TOTAL_FRAMES = VIDEO_INFO.total_frames


In [ ]:
class PolygonZone:
    def __init__(
        self,
        polygon: np.ndarray,
        frame_resolution_wh: Tuple[int, int],
        triggering_position: Position = Position.BOTTOM_CENTER,
    ):
        self.polygon = polygon
        self.frame_resolution_wh = frame_resolution_wh
        self.triggering_position = triggering_position
        self.mask = generate_2d_mask(
            polygon=polygon, resolution_wh=frame_resolution_wh)
        self.current_count = 0

    def trigger(self, detections, data) -> np.ndarray:
        anchors = (
            np.ceil(
                detections.get_anchor_coordinates(
                    anchor=self.triggering_position)
            ).astype(int)
            - 1
        )
        is_in_zone = self.mask[anchors[:, 1], anchors[:, 0]]
        self.current_count = np.sum(is_in_zone)
        data.append(self.current_count)
        return is_in_zone.astype(bool),self.current_count


class PolygonZoneAnnotator:
    def __init__(
        self,
        zone: PolygonZone,
        color: Color,
        thickness: int = 2,
        text_color: Color = Color.blue(),
        text_scale: float = 0.5,
        text_thickness: int = 1,
        text_padding: int = 10,
    ):
        self.zone = zone
        self.color = color
        self.thickness = thickness
        self.text_color = text_color
        self.text_scale = text_scale
        self.text_thickness = text_thickness
        self.text_padding = text_padding
        self.font = cv2.FONT_HERSHEY_SIMPLEX
        self.center = get_polygon_center(polygon=zone.polygon)

    def annotate(self, totalNumber: Optional[int], scene: np.ndarray, label: Optional[str] = None) -> np.ndarray:
        annotated_frame = draw_polygon(
            scene=scene,
            polygon=self.zone.polygon,
            color=self.color,
            thickness=self.thickness,
        )

        annotated_frame = draw_text(
            scene=annotated_frame,
            # text=("") if label is None else label,
            text=(
                f"Current People In Zone: {self.zone.current_count}") if label is None else label,
            text_anchor=Point(450, 100),
            background_color=self.color,
            text_color=self.text_color,
            text_scale=self.text_scale,
            text_thickness=self.text_thickness,
            text_padding=self.text_padding,
            text_font=self.font,
        )
        if "." in str(totalNumber):
            totalNumber = str(totalNumber)
            totalNumber = totalNumber[:len(totalNumber)-2]
        annotated_frame = draw_text(
            scene=annotated_frame,
            # text=("") if label is None else label,
            text=(
                f"Total People In Zone: {totalNumber}") if label is None else label,
            text_anchor=Point(450, 250),
            background_color=self.color,
            text_color=self.text_color,
            text_scale=self.text_scale,
            text_thickness=self.text_thickness,
            text_padding=self.text_padding,
            text_font=self.font,
        )
        return annotated_frame


In [ ]:

personPerFrame = [0, 0]
res = 0
N = 40
personPerNFrame = [0, 0]
i = 0
sumPerson = 0


def process_frame(frame: np.ndarray, _) -> np.ndarray:
    # detect
    results = model(frame, imgsz=1280)[0]
    detections = sv.Detections.from_yolov8(results)
    detections = detections[(detections.class_id == 0)
                            & (detections.confidence > 0.4)]
    zone.trigger(detections,personPerFrame)

    # annotate
    box_annotator = sv.BoxAnnotator(
        thickness=4, text_thickness=4, text_scale=2)
    labels = [f"{model.names[class_id]} {confidence:0.2f}" for _,
              confidence, class_id, _ in detections]
    frame = box_annotator.annotate(
        scene=frame, detections=detections, labels=labels)
    frame = zone_annotator.annotate(res, scene=frame,)

    return frame


# initiate polygon zone
polygon = np.array([
    [961, 1078],
    [763, 550],
    [1502, 254],
    [1914, 260],
    [1920, 1080]
])
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

zone = PolygonZone(
    polygon=polygon, frame_resolution_wh=video_info.resolution_wh)

# initiate annotators
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
zone_annotator = PolygonZoneAnnotator(
    zone=zone, color=sv.Color.white(), thickness=6, text_thickness=3, text_scale=2)


generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

iterator = iter(generator)
frame = next(iterator)

out = cv2.VideoWriter(TARGET_VIDEO_PATH, cv2.VideoWriter_fourcc(
    *'mp4v'), FPS, (WIDTH//2, HEIGHT//2))
num = 0

for frame in iterator:
    num += 1
    current = process_frame(frame, 1)

    if i < N:
        sumPerson += personPerFrame[-1]
        i += 1
    else:
        numPerson = sumPerson/N
        if numPerson-numPerson//1 < 0.5:
            personPerNFrame.append(numPerson//1)
        else:
            personPerNFrame.append(numPerson//1+1)
        sumPerson = 0
        i = 0

        currentFrame = personPerNFrame[-1]
        previousFrame = personPerNFrame[-2]
        if currentFrame < previousFrame:
            res = max(res, previousFrame)
        else:
            res += currentFrame-previousFrame
    print(f"TOTAL NUMBER: {res}")

    width = int(current.shape[1] * 0.5)
    height = int(current.shape[0] * 0.5)
    new_size = (width, height)

    resized_img = cv2.resize(current, new_size)
    out.write(resized_img)
    # cv2.imshow('Pedestrian Detection', resized_img)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
        # break
out.release()
cv2.destroyAllWindows()

